# Satellite Image Feature Matching with SuperPoint and SuperGlue

This script performs feature matching between pairs of satellite images using the SuperPoint and SuperGlue models. It detects and matches keypoints between images, which can be useful for monitoring environmental changes such as deforestation.

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
from glob import glob
from torchvision import transforms

## Step 1: Install and Import Required Libraries
Install necessary libraries for image processing and deep learning and import them.

In [12]:
!pip install --upgrade torch torchvision kornia 

  Using cached torch-2.5.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.20.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 964.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 825.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2: Clone and Set Up SuperGlue Repository
Clone the SuperGlue repository and add it to the system path to access the necessary models and utilities.

In [3]:
!git clone https://github.com/magicleap/SuperGluePretrainedNetwork.git
import sys
sys.path.append('./SuperGluePretrainedNetwork')

Cloning into 'SuperGluePretrainedNetwork'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 185 (delta 0), reused 2 (delta 0), pack-reused 182 (from 1)
Receiving objects: 100% (185/185), 118.85 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [17]:
!pip install supergluepy

ERROR: Could not find a version that satisfies the requirement supergluepy (from versions: none)
ERROR: No matching distribution found for supergluepy


## Step 3: Import Matching Modules
Import the `Matching` class and utility functions for reading images and visualizing matches.

In [19]:
from models.matching import Matching
from models.utils import read_image, make_matching_plot

## Step 4: Define Helper Functions
Define helper functions to locate pairs of satellite images and load them. These functions organize images by date and band, making them ready for feature matching.

In [21]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 61.1 MB/s eta 0:00:00:00:0100:01


In [22]:
import rasterio

def find_image_pairs(data_dir):
    images_by_tile = {}
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jp2'):
                filepath = os.path.join(root, file)
                filename = os.path.basename(filepath)
                parts = filename.split('_')
                if len(parts) >= 3:
                    tile = parts[0]
                    date = parts[1][:8]  # YYYYMMDD
                    band_part = parts[-1]
                    band = band_part.split('.')[0]
                    key = (tile, date)
                    if key not in images_by_tile:
                        images_by_tile[key] = {}
                    images_by_tile[key][band] = filepath

    tile_dates = {}
    for (tile, date), bands in images_by_tile.items():
        if tile not in tile_dates:
            tile_dates[tile] = []
        tile_dates[tile].append((date, bands))
    for tile in tile_dates:
        tile_dates[tile].sort()

    image_pairs = []
    for tile in tile_dates:
        dates_bands = tile_dates[tile]
        for i in range(len(dates_bands) - 1):
            date1, bands1 = dates_bands[i]
            date2, bands2 = dates_bands[i+1]
            required_bands = {'B02', 'B03', 'B04'}
            if required_bands.issubset(bands1.keys()) and required_bands.issubset(bands2.keys()):
                image_pairs.append(((bands1['B02'], bands1['B03'], bands1['B04']),
                                    (bands2['B02'], bands2['B03'], bands2['B04'])))
    return image_pairs

def load_sentinel2_image(band_paths):
    try:
        with rasterio.open(band_paths[0]) as blue_src, \
             rasterio.open(band_paths[1]) as green_src, \
             rasterio.open(band_paths[2]) as red_src:
            blue = blue_src.read(1)
            green = green_src.read(1)
            red = red_src.read(1)
            image = np.stack((red, green, blue), axis=-1)
            image_gray = cv2.cvtColor((image / np.max(image) * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
            return image_gray, image
    except Exception as e:
        print(f'Error loading bands: {e}')
        return None, None

## Step 5: Load and Preprocess Images
Identify image pairs, load one pair, and resize them to a standard size for processing.

In [23]:
data_dir = '/kaggle/input/deforestation-in-ukraine'
image_pairs = find_image_pairs(data_dir)
(band_paths1), (band_paths2) = image_pairs[5]
image0_gray, image0_color = load_sentinel2_image(band_paths1)
image1_gray, image1_color = load_sentinel2_image(band_paths2)
image0_gray = cv2.resize(image0_gray, (640, 480))
image1_gray = cv2.resize(image1_gray, (640, 480))

## Step 6: Configure and Run the Matching Model
Configure SuperPoint and SuperGlue, then perform feature matching on the selected image pair.

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': 1024,
    },
    'superglue': {
        'weights': 'outdoor',
        'sinkhorn_iterations': 20,
        'match_threshold': 0.2,
    }
}
matching = Matching(config).eval().to(device)

inp0 = torch.from_numpy(image0_gray / 255.).float()[None, None].to(device)
inp1 = torch.from_numpy(image1_gray / 255.).float()[None, None].to(device)
with torch.no_grad():
    pred = matching({'image0': inp0, 'image1': inp1})
keypoints0 = pred['keypoints0'][0].cpu().numpy()
keypoints1 = pred['keypoints1'][0].cpu().numpy()
matches = pred['matches0'][0].cpu().numpy()
confidence = pred['matching_scores0'][0].cpu().numpy()

/kaggle/working/./SuperGluePretrainedNetwork/models/superpoint.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str(path)))
/kaggle/work

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


## Step 7: Filter and Visualize Matches
Filter the valid matches and visualize the matched keypoints.

In [26]:
valid = matches > -1
mkpts0 = keypoints0[valid]
mkpts1 = keypoints1[matches[valid]]
mconf = confidence[valid]

color = plt.cm.jet(mconf)
make_matching_plot(
    cv2.cvtColor(image0_color, cv2.COLOR_RGB2GRAY),
    cv2.cvtColor(image1_color, cv2.COLOR_RGB2GRAY),
    keypoints0, keypoints1, mkpts0, mkpts1, color,
    text=['SuperGlue Feature Matching', f'Keypoints: {len(keypoints0)}:{len(keypoints1)}', f'Matches: {len(mkpts0)}'],
    path='matches.png', show_keypoints=True, opencv_display=False
)
plt.figure(figsize=(20, 10))
matched_img = plt.imread('matches.png')
plt.imshow(matched_img, cmap='gray')
plt.title('Keypoint Matches Between Images Using SuperPoint and SuperGlue')
plt.axis('off')
plt.show()

## Conclusions and Possible Improvements

- **Dataset Diversity**: This setup currently handles a specific format and a single data source. Expanding to include additional satellite sources or image types could make the model more robust across datasets.
- **Error Handling**: Improved error handling in the `load_sentinel2_image` function would enhance reliability, particularly for large-scale batch processing.
- **Hyperparameter Tuning**: Fine-tuning model parameters like `keypoint_threshold` and `match_threshold` could yield more accurate results.
- **RANSAC Filtering**: Adding RANSAC (Random Sample Consensus) could filter outliers, improving match quality for specific applications.
- **Scalability**: To handle larger datasets, consider batch processing or parallel processing to improve efficiency.